In [1]:
#imports
import requests
import pandas as pd
import json
import time

In [2]:
class Release_Node:
    """
    Represents a node in a networkx diagram.with basic attributes and functionalities(TBD).

    Attributes:
        release_name (str): Name of the music release.
        release_mbid (str): MusicBrainz ID.
        release_credits (list): List of credits on the music release.
    """
    def __init__(self, release_name, release_data, release_credits):
        """
        Initializes the Release_Node object.

        Args:
            release_name (str): Name of the music release.
            release_mbid (str): MusicBrainz ID.
            release_credits (list): List of credits on the music release.
        """
        self.release_name = name
        self.release_mbid = data
        self.release_credits = []

In [3]:
def search_recording(artist_name, release_title):
    """
    Retrieves all release groups for a given artist using their MusicBrainz ID.

    Args:
        artist_name (str): Name of the artist.
        release_title (str): Name of the Release

    Returns:
        list: The MBID for the release. This is the initial node for the network.
    """
    base_url = "https://musicbrainz.org/ws/2/release/"
    params = {
        "query": f"artist:\"{artist_name}\" AND release:\"{release_title}\"",
        "fmt": "json"
    }
    headers = {"User-Agent": "Mozilla/5.0 (bittah.pupil@proton.me)"} # Replace with your app info

    response = requests.get(base_url, params=params, headers=headers)
    response.raise_for_status() # Raise an exception for bad status codes
    data = response.json()
    
    if data and data.get('releases'):
        # Return the MBID of the first matching recording
        return data['releases'][0]['id']
    return None

# Example usage:
artist = "Steely Dan"
album = "Everything Must Go"
release_mbid = search_recording(artist, album)
print(f"Recording MBID for '{album}' by '{artist}': {release_mbid}")

Recording MBID for 'Everything Must Go' by 'Steely Dan': 6c54186c-ba4c-4391-9f6f-b8521c7b11d0


In [4]:
async def get_credits_for_recording(release_mbid):
    """
    Retrieves all the credits for a given release using the releases MusicBrainz ID.

    Args:
        release_mbid (str): MusicBrainz ID for the release.

    Returns:
        list: A list of all credited personnel (including artist) on the release based on the provided release mbid.
        This list includes the name of the credit, the role of the credit, and the mbid for the credit.
    """
    base_url = f"https://musicbrainz.org/ws/2/release/{release_mbid}"
    params = {
        "inc": "artist-credits+labels+recordings+recording-level-rels+work-rels+work-level-rels+artist-rels + release-groups",  # Include artist relationships
        "fmt": "json"
    }
    headers = {"User-Agent": "Mozilla/5.0 (bittah.pupil@proton.me)"}

    response = requests.get(base_url, params=params, headers=headers)
    response.raise_for_status()
    data = response.json()

    credits = []
    
    if data and data.get('artist-credit'):
       for track in data['media'][0]['tracks']:
            for credit in track['recording']['relations']:
                if credit['attributes'] == []:
                    attributes = 'Not Found'
                else:
                    attributes = credit['attributes']
                credits.append((track['number'], track['title'], credit.get('artist', {}).get('name', 'Not Found'), attributes, credit.get('artist', {}).get('id', 'Not Found')))
            for credit in track['artist-credit']:
                credits.append((track['number'], track['title'], credit.get('name', 'Not Found'), credit.get('attributes', 'Not Found'), credit.get('artist', {}).get('id', 'Not Found')))
    if data['relations']:
       for credit in data['relations']:
           trackNum = 'Not Found'
           trackTitle = 'Not Found'
           if credit['attributes'] == [] or not credit['attributes']:
               attributes = 'Not Found'
           else:
               attributes = credit['attributes']
           credits.append((trackNum, trackTitle, credit.get('artist', {}).get('name', 'Not Found'), attributes, credit.get('artist', {}).get('id', 'Not Found')))

    creditNames = []
    creditList = []
    for credit in credits:
        if credit[2] not in creditNames:
            creditNames.append(credit[2])
            creditList.append((credit[2],credit[3], credit[4]))
    print(creditNames)
    print("\n")
    print(creditList)

    artist_mbid = creditList[0][2]
    print(artist_mbid)
    
    return credits

In [5]:
#Example usage:
if release_mbid:
    albumCredits = await get_credits_for_recording(release_mbid)
    #print(albumCredits)
else:
    print(f"Recording for '{album}' by '{artist}' not found.")

time.sleep(2)

# Create the DataFrame
df = pd.DataFrame(albumCredits, columns = ['Track Number', 'Track Title', 'Artist', 'Role', 'Artist_MBID'])

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)


# Display the DataFrame
df

['Ted Baker', 'Walter Becker', 'Keith Carlock', 'Donald Fagen', 'Jon Herington', 'Tony Kadleck', 'Hugh McCracken', 'Jim Pugh', 'Roger Rosenberg', 'Walt Weiskopf', 'Joe Peccerillo', 'Elliot Scheiner', 'Cindy Mizelle', 'Catherine Russell', 'Not Found', 'Steely Dan', 'Gordon Gottlieb', 'Michael Leonhart', 'Carolyn Leonhart', 'Bill Charlap', 'Tawatha Agee', 'Ada Dyer', 'Michael Harvey', 'Ken Hitchcock', 'Chris Potter', 'Brenda White‐King']


[('Ted Baker', ['piano'], 'ffd99695-e69f-4d8a-9905-ab44801349a2'), ('Walter Becker', ['electric bass guitar', 'electric guitar', 'solo'], '8cc72457-488e-44e6-a5b3-d2da843b7245'), ('Keith Carlock', ['drums (drum set)'], '318c5217-5b77-46aa-8662-b7f2aa8f7774'), ('Donald Fagen', ['Rhodes piano', 'Wurlitzer electric piano'], '70047e57-0153-4117-b0fc-a1d2e322e5ef'), ('Jon Herington', ['electric guitar'], '94ad5b05-7843-44ce-b994-b17830478f7d'), ('Tony Kadleck', ['trumpet'], '24c1fe59-117b-4c66-8803-e322dc8574a4'), ('Hugh McCracken', ['electric guitar'], 'fd

,Track Number,Track Title,Artist,Role,Artist_MBID
0,1,The Last Mall,Ted Baker,[piano],ffd99695-e69f-4d8a-9905-ab44801349a2
1,1,The Last Mall,Walter Becker,"[electric bass guitar, electric guitar, solo]",8cc72457-488e-44e6-a5b3-d2da843b7245
2,1,The Last Mall,Keith Carlock,[drums (drum set)],318c5217-5b77-46aa-8662-b7f2aa8f7774
3,1,The Last Mall,Donald Fagen,"[Rhodes piano, Wurlitzer electric piano]",70047e57-0153-4117-b0fc-a1d2e322e5ef
4,1,The Last Mall,Jon Herington,[electric guitar],94ad5b05-7843-44ce-b994-b17830478f7d
5,1,The Last Mall,Tony Kadleck,[trumpet],24c1fe59-117b-4c66-8803-e322dc8574a4
6,1,The Last Mall,Hugh McCracken,[electric guitar],fd7ce65a-3eea-4bec-9b05-7c01d43b3b00
7,1,The Last Mall,Jim Pugh,[trombone],fd1262b1-b80c-4836-a167-0c2d7420b110
8,1,The Last Mall,Roger Rosenberg,[baritone saxophone],6dd659ef-d184-405a-afb5-ed73ea27ab51
9,1,The Last Mall,Walt Weiskopf,[alto saxophone],81f5f851-bf90-4e97-940a-c1e87785899c
